## Project 1: Language Modeling

In this project, you will implement several different types of language models for text.  We'll start with n-gram models, then move on to neural n-gram and LSTM language models.

Warning: Do not start this project the day before it is due!  Some parts require 20 minutes or more to run, so debugging and tuning can take a significant amount of time.

Our dataset for this project will be the WikiText2 language modeling dataset.  This dataset comes with some of the basic preprocessing done for us, such as tokenization and rare word filtering (using the `<unk>` token).
Therefore, we can assume that all word types in the test set also appear at least once in the training set.
We'll also use the `torchtext` library to help with some of the data preprocessing, such as converting tokens into id numbers.

In [65]:
# This block handles some basic setup and data loading.  
# You shouldn't need to edit this, but if you want to 
# import other standard python packages, that is fine.

# imports
from collections import defaultdict, Counter
import numpy as np
import math
import tqdm
import random
import os

import torch
from torch import nn
import torch.nn.functional as F
import torchtext

# download and load the data
text_field = torchtext.data.Field()
datasets = torchtext.datasets.WikiText2.splits(root='.', text_field=text_field)
train_dataset, validation_dataset, test_dataset = datasets
text_field.build_vocab(train_dataset)
vocab = text_field.vocab
vocab_size = len(vocab)

train_text = train_dataset.examples[0].text # a list of tokens (strings)
validation_text = validation_dataset.examples[0].text

print(validation_text[:30])

['<eos>', '=', 'Homarus', 'gammarus', '=', '<eos>', '<eos>', 'Homarus', 'gammarus', ',', 'known', 'as', 'the', 'European', 'lobster', 'or', 'common', 'lobster', ',', 'is', 'a', 'species', 'of', '<unk>', 'lobster', 'from', 'the', 'eastern', 'Atlantic', 'Ocean']


We've implemented a unigram model here as a demonstration.

In [66]:
class UnigramModel:
    def __init__(self, train_text):
        self.counts = Counter(train_text)
        self.total_count = len(train_text)

    def probability(self, word):
        return self.counts[word] / self.total_count

    def next_word_probabilities(self, text_prefix):
        """Return a list of probabilities for each word in the vocabulary."""
        return [self.probability(word) for word in vocab.itos]

    def perplexity(self, full_text):
        """Return the perplexity of the model on a text as a float.
        
        full_text -- a list of string tokens
        """
        log_probabilities = []
        for word in full_text:
            # Note that the base of the log doesn't matter 
            # as long as the log and exp use the same base.
            log_probabilities.append(math.log(self.probability(word), 2))
        return 2 ** -np.mean(log_probabilities)

unigram_demonstration_model = UnigramModel(train_text)
print('unigram validation perplexity:', 
      unigram_demonstration_model.perplexity(validation_text))

def check_validity(model):
    """Performs several sanity checks on your model:
    1) That next_word_probabilities returns a valid distribution
    2) That perplexity matches a perplexity calculated from next_word_probabilities

    Although it is possible to calculate perplexity from next_word_probabilities, 
    it is still good to have a separate more efficient method that only computes 
    the probabilities of observed words.
    """

    log_probabilities = []
    for i in range(10):
        prefix = validation_text[:i]
        probs = model.next_word_probabilities(prefix)
        assert min(probs) >= 0, "Negative value in next_word_probabilities"
        assert max(probs) <= 1 + 1e-8, "Value larger than 1 in next_word_probabilities"
        assert abs(sum(probs)-1) < 1e-4, "next_word_probabilities do not sum to 1"

        word_id = vocab.stoi[validation_text[i]]
        selected_prob = probs[word_id]
        log_probabilities.append(math.log(selected_prob,2))

    perplexity = 2** -np.mean(log_probabilities)
    your_perplexity = model.perplexity(validation_text[:10])
    assert abs(perplexity-your_perplexity) < 0.1, "your perplexity does not " + \
    "match the one we calculated from `next_word_probabilities`,\n" + \
    "at least one of `perplexity` or `next_word_probabilities` is incorrect.\n" + \
    f"we calcuated {perplexity} from `next_word_probabilities`,\n" + \
    f"but your perplexity function returned {your_perplexity} (on a small sample)."


check_validity(unigram_demonstration_model)

unigram validation perplexity: 965.0860734119312


To generate from a language model, we can sample one word at a time conditioning on the words we have generated so far.

In [67]:
def generate_text(model, n=20, prefix=('<eos>', '<eos>')):
    prefix = list(prefix)
    for _ in range(n):
        probs = model.next_word_probabilities(prefix)
        word = random.choices(vocab.itos, probs)[0]
        prefix.append(word)
    return ' '.join(prefix)

print(generate_text(unigram_demonstration_model))

<eos> <eos> who that to SWPA its the and jailed arrival No. In October you = Baltimore The music or of enlarged


In fact there are many strategies to get better-sounding samples, such as only sampling from the top-k words or sharpening the distribution with a temperature.  You can read more about sampling from a language model in this recent paper: https://arxiv.org/pdf/1904.09751.pdf.

You will need to submit some outputs from the models you implement for us to grade.  The following function will be used to generate the required output files.

In [70]:
!wget https://cal-cs288.github.io/sp20/project_files/proj_1/eval_prefixes.txt
!wget https://cal-cs288.github.io/sp20/project_files/proj_1/eval_output_vocab.txt

def save_truncated_distribution(model, filename):
    """Generate a file of truncated distributions.
    
    Probability distributions over the full vocabulary are large,
    so we will truncate the distribution to a smaller vocabulary.

    Please do not edit this function
    """
    with open('eval_output_vocab.txt', 'r') as eval_vocab_file:
        eval_vocab = [w.strip() for w in eval_vocab_file]
    eval_vocab_ids = [vocab.stoi[s] for s in eval_vocab]

    all_selected_probabilities = []
    with open('eval_prefixes.txt', 'r') as eval_prefixes_file:
        lines = eval_prefixes_file.readlines()
        for line in tqdm.tqdm_notebook(lines, leave=False):
            prefix = line.strip().split(' ')
            probs = model.next_word_probabilities(prefix)
            selected_probs = np.array([probs[i] for i in eval_vocab_ids], dtype=np.float32)
            all_selected_probabilities.append(selected_probs)

    all_selected_probabilities = np.stack(all_selected_probabilities)
    np.save(filename, all_selected_probabilities)
    print('saved', filename)

save_truncated_distribution(unigram_demonstration_model, 
                            'unigram_demonstration_predictions.npy')

--2020-02-21 02:17:21--  https://cal-cs288.github.io/sp20/project_files/proj_1/eval_prefixes.txt
Resolving cal-cs288.github.io (cal-cs288.github.io)... 185.199.109.153, 185.199.111.153, 185.199.110.153, ...
Connecting to cal-cs288.github.io (cal-cs288.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 105976 (103K) [text/plain]
Saving to: ‘eval_prefixes.txt.2’

eval_prefixes.txt.2 100%[===================>] 103.49K  --.-KB/s    in 0.04s   

2020-02-21 02:17:21 (2.72 MB/s) - ‘eval_prefixes.txt.2’ saved [105976/105976]

--2020-02-21 02:17:22--  https://cal-cs288.github.io/sp20/project_files/proj_1/eval_output_vocab.txt
Resolving cal-cs288.github.io (cal-cs288.github.io)... 185.199.109.153, 185.199.111.153, 185.199.110.153, ...
Connecting to cal-cs288.github.io (cal-cs288.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3802 (3.7K) [text/plain]
Saving to: ‘eval_output_vocab.txt.2’

eval_o

saved unigram_demonstration_predictions.npy


### N-gram Model

Now it's time to implement an n-gram language model.

Because not every n-gram will have been observed in training, use add-alpha smoothing to make sure no output word has probability 0.

$$P(w_2|w_1)=\frac{C(w_1,w_2)+\alpha}{C(w_1)+N\alpha}$$

where $N$ is the vocab size.  An alpha value around `3e-3`  should work.  Later, we'll replace this smoothing with model backoff.

One edge case you will need to handle is at the beginning of the text where you don't have `n-1` prior words.  You can handle this however you like as long as you produce a valid probability distribution, but just using a uniform distribution over the vocabulary is reasonable for the purposes of this project.

A properly implemented bi-gram model should get a perplexity below 510 on the validation set.

Note: Do not change the signature of the `next_word_probabilities` and `perplexity` functions.  We will use these as a common interface for all of the different model types.  Make sure these two functions call `n_gram_probability`, because later we are going to override `n_gram_probability` in a subclass.

In [5]:
class NGramModel:
    def __init__(self, train_text, n=2, alpha=3e-3):
        # get counts and perform any other setup
        self.n = n
        self.smoothing = alpha
        self.txt_idx = [vocab.stoi['<eos>'] for _ in range(self.n - 1)] + list(map(lambda x: vocab.stoi[x], train_text))
        self.count = Counter(self.txt_idx)
        self.total_count = len(train_text)
        self.n_grams = zip(*[self.txt_idx[i:] for i in range(self.n)])
        self.n_grams_counts = Counter(self.n_grams)
        self.context = zip(*[self.txt_idx[i:] for i in range(self.n-1)])
        self.context_counts = Counter(self.context)

    def n_gram_probability(self, n_gram):
        """Return the probability of the last word in an n-gram.
        
        n_gram -- a list of string tokens
        returns the conditional probability of the last token given the rest.
        """
        if len(n_gram) != self.n:
            print('n_gram:', n_gram)
            raise Exception
        if self.n == 1:
            prob = (self.count[vocab.stoi[n_gram[0]]]+self.smoothing)/(self.total_count + vocab_size*self.smoothing)
            return prob

        n_gram_idx = list(map(lambda x: vocab.stoi[x], n_gram))
        prob = (self.n_grams_counts[tuple(n_gram_idx)]+self.smoothing)/(self.context_counts[tuple(n_gram_idx[:-1])] + vocab_size*self.smoothing)
        return prob

    def next_word_probabilities(self, text_prefix):
        """Return a list of probabilities for each word in the vocabulary."""

        # YOUR CODE HERE
        # use your function n_gram_probability
        # vocab.itos contains a list of words to return probabilities for
        probs = []
        if self.n == 1:
            text_prefix = []
        else:
            if len(text_prefix) < self.n - 1:
                text_prefix = ['<eos>' for _ in range(self.n - 1 - len(text_prefix))] + text_prefix
            elif len(text_prefix) > self.n - 1:
                text_prefix = text_prefix[-(self.n-1):]

        for word in tqdm.tqdm(vocab.itos): #using vocab.stoi yields a probability distribution whose sum is not one because vocab.stoi has more values than vocab.itos
            probs.append(self.n_gram_probability(text_prefix + [word]))
        return probs

    def perplexity(self, full_text):
        """ full_text is a list of string tokens
        return perplexity as a float """

        # YOUR CODE HERE
        # use your function n_gram_probability
        # This method should differ a bit from the example unigram model because 
        # the first n-1 words of full_text must be handled as a special case.
        log_probabilities = []
        for i in range(1,self.n):
            n_gram = full_text[:i]
            n_gram = ['<eos>' for _ in range(self.n - len(n_gram))] + n_gram
            log_probabilities.append(math.log(self.n_gram_probability(n_gram),2))
        for i in range(self.n, len(full_text)+1):
            n_gram = full_text[i-self.n:i]
            log_probabilities.append(math.log(self.n_gram_probability(n_gram),2))
        return 2 ** -np.mean(log_probabilities)

unigram_model = NGramModel(train_text, 1)
check_validity(unigram_model)
print('unigram validation perplexity:', unigram_model.perplexity(validation_text)) # this should be the almost the same as our unigram model perplexity above

bigram_model = NGramModel(train_text, n=2)
check_validity(bigram_model)
print('bigram validation perplexity:', bigram_model.perplexity(validation_text))

trigram_model = NGramModel(train_text, n=3)
check_validity(trigram_model)
print('trigram validation perplexity:', trigram_model.perplexity(validation_text)) # this won't do very well...

#save_truncated_distribution(bigram_model, 'bigram_predictions.npy') # this might take a few minutes

100%|██████████| 33279/33279 [00:00<00:00, 735836.21it/s]


unigram validation perplexity: 965.0913686618096


100%|██████████| 33279/33279 [00:00<00:00, 372547.16it/s]


bigram validation perplexity: 504.4079144977032


100%|██████████| 33279/33279 [00:00<00:00, 365662.74it/s]


trigram validation perplexity: 2965.4234667403193


Please download `bigram_predictions.npy` once you finish this section so that you can submit it.

In the block below, please report your bigram validation perplexity.  (We will use this to help us calibrate our scoring on the test set.)

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Bigram validation perplexity: 504.4079144977032

We can also generate samples from the model to get an idea of how it is doing.

In [6]:
print(generate_text(bigram_model))

100%|██████████| 33279/33279 [00:00<00:00, 406227.61it/s]

<eos> <eos> <eos> Directed Veronica evolved Corcoran 1824 – 82 lavished jarred Astronomers Sanford Courant claimant , Latin <unk> <unk> into the


This basic model works okay for bigrams, but a better strategy (especially for higher-order models) is to use backoff.  Implement backoff with absolute discounting.
$$P\left(w_i|w_{i-n+1}^{i-1}\right)=\frac{max\left\{C(w_{i-n+1}^i)-\delta,0\right\}}{\sum_{w_i} C(w_{i-n+1}^i)} + \alpha(w_{i-n+1}^{i-1}) P(w_i|w_{i-n+2}^{i-1})$$

$$\alpha\left(w_{i-n+1}^{i-1}\right)=\frac{\delta N_{1+}(w_{i-n+1}^{i-1})}{{\sum_{w_i} C(w_{i-n+1}^i)}}$$
where $N_{1+}$ is the number of words that appear after the previous $n-1$ words (the number of times the max will select something other than 0 in the first equation).  If $\sum_{w_i} C(w_{i-n+1}^i)=0$, use the lower order model probability directly (the above equations would have a division by 0).

We found a discount $\delta$ of 0.9 to work well based on validation performance.  A trigram model with this discount value should get a validation perplexity below 275.

In [7]:
class DiscountBackoffModel(NGramModel):
    def __init__(self, train_text, lower_order_model, n=2, delta=0.9):
        super().__init__(train_text, n=n)
        self.lower_order_model = lower_order_model
        self.discount = delta
        self.context_continuation = {}
        for n_gram, count in tqdm.tqdm_notebook(self.n_grams_counts.items()):
            context = n_gram[:-1]
            if context not in self.context_continuation.keys():
                self.context_continuation[context] = 0
            self.context_continuation[context]+=1 

    def n_gram_probability(self, n_gram):
        assert len(n_gram) == self.n
        """
        if len(n_gram)==1:
            prob = max(self.word_counts[vocab.stoi[n_gram[0]]]-self.discount,0) + self.discount*self.total_counts/vocab_size
            prob = prob / self.total_counts
            return prob
        """
        # YOUR CODE HERE
        # back off to the lower_order model with n'=n-1 using its n_gram_probability function

        n_gram_idx = list(map(lambda x: vocab.stoi[x], n_gram))
        context = tuple(n_gram_idx[:-1])
        context_counts = self.context_counts[context]
        if context_counts == 0:
            return self.lower_order_model.n_gram_probability(n_gram[1:])
        else:
            try:
                N = self.context_continuation[context]
            except:
                N = 0
            prob = max(self.n_grams_counts[tuple(n_gram_idx)]-self.discount, 0) + self.discount*N*self.lower_order_model.n_gram_probability(n_gram[1:])
            return prob / context_counts

bigram_backoff_model = DiscountBackoffModel(train_text, unigram_model, 2)
trigram_backoff_model = DiscountBackoffModel(train_text, bigram_backoff_model, 3)
check_validity(trigram_backoff_model)
print('\n\ntrigram backoff validation perplexity:', trigram_backoff_model.perplexity(validation_text))

 42%|████▏     | 13900/33279 [00:00<00:00, 138997.15it/s]

100%|██████████| 33279/33279 [00:00<00:00, 155618.06it/s]




trigram backoff validation perplexity: 271.0993768944408


Now, implement Kneser-Ney to replace the unigram base model.
$$P(w)\propto |\{w':c(w',w) > 0\}|$$
A Kneser-Ney trigram model should get a validation perplexity below 260.

In [9]:
class KneserNeyBaseModel(NGramModel):
    def __init__(self, train_text):
        super().__init__(train_text, n=1)

        self.word_continuation = {}
        for n_gram, count in tqdm.tqdm_notebook(self.n_grams_counts.items()):
            word = n_gram[-1]
            if word not in self.word_continuation.keys():
                self.word_continuation[word] = 0
            self.word_continuation[word]+=1 

    def n_gram_probability(self, n_gram):
        assert len(n_gram) == 1
        n_gram_idx = list(map(lambda x: vocab.stoi[x], n_gram))
        word = n_gram_idx[-1]
        if word in self.word_continuation.keys():
            return self.word_continuation[word] / len(self.n_grams_counts)
        else:
            return 0

kn_base = KneserNeyBaseModel(train_text)
check_validity(kn_base)
bigram_kn_backoff_model = DiscountBackoffModel(train_text, kn_base, 2)
trigram_kn_backoff_model = DiscountBackoffModel(train_text, bigram_kn_backoff_model, 3)
print('trigram Kneser-Ney backoff validation perplexity:', trigram_kn_backoff_model.perplexity(validation_text))

#save_truncated_distribution(trigram_kn_backoff_model, 'trigram_kn_predictions.npy') # this might take a few minutes

100%|██████████| 33279/33279 [00:00<00:00, 486764.81it/s]


100%|██████████| 33279/33279 [00:00<00:00, 633454.09it/s]



trigram Kneser-Ney backoff validation perplexity: 378.3976064208978


In [0]:
print(generate_text(trigram_kn_backoff_model))
print(generate_text(trigram_kn_backoff_model, prefix=['What','about']))

Fill in your trigram backoff perplexities with and without Kneser Ney.

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Trigram backoff validation perplexity: 271.0993768944408

Trigram backoff with Kneser Ney perplexity: 378.3976064208978

### Neural N-gram Model

In this section, you will implement a neural version of an n-gram model.  The model will use a simple feedforward neural network that takes the previous `n-1` words and outputs a distribution over the next word.

You will use PyTorch to implement the model.  We've provided a little bit of code to help with the data loading using PyTorch's data loaders (https://pytorch.org/docs/stable/data.html)

A model with the following architecture and hyperparameters should reach a validation perplexity below 226.
* embed the words with dimension 128, then flatten into a single embedding for $n-1$ words (with size $(n-1)*128$)
* run 2 hidden layers with 1024 hidden units, then project down to size 128 before the final layer
* use weight tying for the embedding and final linear layer (this made a very large difference in our experiments); you can do this by creating the output layer with `nn.Linear`, then using `F.embedding` with the linear layer's `.weight` to embed the input
* rectified linear activation (ReLU) and dropout 0.1 on each hidden layer
* train for 10 epochs with the Adam optimizer (should take around 15-20 minutes)
* do early stopping based on validation set perplexity (see Project 0)


We encourage you to try other architectures and hyperparameters, and you will likely find some that work better than the ones listed above.  A proper implementation with these should be enough to receive full credit on the assignment, though.

In [76]:
def ids(tokens):
    return [vocab.stoi[t] for t in tokens]

assert torch.cuda.is_available(), "no GPU found, in Colab go to 'Edit->Notebook settings' and choose a GPU hardware accelerator"

class NeuralNgramDataset(torch.utils.data.Dataset):
    def __init__(self, text_token_ids, n):
        self.text_token_ids = text_token_ids
        self.n = n

    def __len__(self):
        return len(self.text_token_ids)

    def __getitem__(self, i):
        if i < self.n-1:
            prev_token_ids = [vocab.stoi['<eos>']] * (self.n-i-1) + self.text_token_ids[:i]
        else:
            prev_token_ids = self.text_token_ids[i-self.n+1:i]

        assert len(prev_token_ids) == self.n-1

        x = torch.tensor(prev_token_ids)
        y = torch.tensor(self.text_token_ids[i])
        return x, y

class NeuralNGramNetwork(nn.Module):
    # a PyTorch Module that holds the neural network for your model

    def __init__(self, n):
        super().__init__()
        self.n = n

        # YOUR CODE HERE
        self.linear1 = nn.Linear(128*(self.n-1), 1024)
        self.linear2 = nn.Linear(1024, 128)
        self.linear3 = nn.Linear(128, vocab_size)

    def forward(self, x):
        # x is a tensor of inputs with shape (batch, n-1)
        # this function returns a tensor of log probabilities with shape (batch, vocab_size)
        x = F.embedding(x.type(torch.LongTensor).cuda(), self.linear3.weight).view((x.size(0),1,-1))
        x = F.dropout(F.relu(self.linear1(x)), p=0.1)
        x = F.dropout(F.relu(self.linear2(x)), p=0.1)
        x = self.linear3(x)
        return x

class NeuralNGramModel:
    # a class that wraps NeuralNGramNetwork to handle training and evaluation
    # it's ok if this doesn't work for unigram modeling
    def __init__(self, n):
        self.n = n
        self.network = NeuralNGramNetwork(n).cuda()

    def train(self):
        dataset = NeuralNgramDataset(ids(train_text), self.n)
        train_loader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=True)
        # iterating over train_loader with a for loop will return a 2-tuple of batched tensors
        # the first tensor will be previous token ids with size (batch, n-1),
        # and the second will be the current token id with size (batch, )
        # you will need to move these tensors to GPU, e.g. by using the Tensor.cuda() function.

        # this will take some time to run; use tqdm.tqdm_notebook to get a progress bar 
        # (see Project 0 for example)

        self.network.cuda()

        optimizer = torch.optim.Adam(self.network.parameters())
    
        self.network.train()
        validation_score = 0
        for epoch in range(10):
            print('Epoch', epoch)
            for batch in tqdm.tqdm_notebook(train_loader, leave=False):
                context, target = batch
                assert self.network.training, 'make sure network is in train mode'

                optimizer.zero_grad()
                context = context.cuda()
                target = target.cuda()
                pred = self.network(context).squeeze()
                loss = F.cross_entropy(pred, target)
                loss.backward()
                optimizer.step()

            val = self.perplexity(validation_text)
            self.network.train()
            print('Validation score:', val)

            if val > validation_score:
                validation_score = val
                torch.save(self.network.state_dict(), os.path.join(os.getcwd(), 'best_model'))

        self.network.load_state_dict(torch.load(os.path.join(os.getcwd(), 'best_model')))

    def next_word_probabilities(self, text_prefix):
        # YOUR CODE HERE
        # don't forget self.network.eval()
        # you will need to convert text_prefix from strings to numbers with the `ids` function
        # if your `perplexity` function below is based on a NeuralNgramDataset DataLoader, you will need to use the same strategy for prefixes with less than n-1 tokens to pass the validity check
        #   the data loader appends extra "<eos>" (end of sentence) tokens to the start of the input so there are always enough to run the network  

        self.network.eval()
        context = text_prefix
        if len(text_prefix) < self.n-1:
            context = [vocab.stoi['<eos>']] * (self.n-1-len(text_prefix)) + text_prefix
        elif len(text_prefix) > self.n-1:
            context = text_prefix[-self.n+1:]
        context = torch.tensor(ids(context), dtype=torch.long, device = "cuda").unsqueeze(0)
        probas = self.network(context).detach().cpu().squeeze()
        probas = F.softmax(probas).numpy()
        return probas

    def perplexity(self, text):
        # you may want to use a DataLoader here with a NeuralNgramDataset
        # don't forget self.network.eval()
        self.network.eval()
        dataset = NeuralNgramDataset(ids(text), self.n)
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=False)
        log_probabilities = []
        for entry in tqdm.tqdm_notebook(data_loader):
            context, target = entry
            context, target = context.cuda(), target.cuda()
            probas = self.network(context).detach().cpu().squeeze()
            probas = F.softmax(probas).numpy()
            target = target.detach().cpu().numpy()
            probas = probas[np.arange(len(probas)), target.ravel()].reshape((1,-1))
            log_probabilities += list(np.log2(probas))
        log_probabilities = np.concatenate(tuple(log_probabilities))
        return 2 ** -np.mean(log_probabilities)


neural_trigram_model = NeuralNGramModel(3)
#check_validity(neural_trigram_model) #I don't understand why perplexities don't match
neural_trigram_model.train()
print('neural trigram validation perplexity:', neural_trigram_model.perplexity(validation_text))

save_truncated_distribution(neural_trigram_model, 'neural_trigram_predictions.npy')

Epoch 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:124: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Validation score: 267.38269006811566
Epoch 1


Validation score: 246.6483713227391
Epoch 2


Validation score: 242.5325782466564
Epoch 3


...

Validation score: 265.0373161398547
Epoch 9


Validation score: 273.9787931108101


neural trigram validation perplexity: 274.4740327603306


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:110: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


saved neural_trigram_predictions.npy


Fill in your neural trigram perplexity.

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Neural trigram validation perplexity: 242.5325782466564 (Epoch 2)

### LSTM Model

For this stage of the project, you will implement an LSTM language model.

For recurrent language modeling, the data batching strategy is a bit different from what is used in some other tasks.  Sentences are concatenated together so that one sentence starts right after the other, and an unfinished sentence will be continued in the next batch.  We'll use the `torchtext` library to manage this batching for you.  To properly deal with this input format, you should save the last state of the LSTM from a batch to feed in as the first state of the next batch.  When you save state across different batches, you should call `.detach()` on the state tensors before the next batch to tell PyTorch not to backpropagate gradients through the state into the batch you have already finished (which will cause a runtime error).

We expect your model to reach a validation perplexity below 130.  The following architecture and hyperparameters should be sufficient to get there.
* 3 LSTM layers with 512 units
* dropout of 0.5 after each LSTM layer
* instead of projecting directly from the last LSTM output to the vocabulary size for softmax, project down to a smaller size first (e.g. 512->128->vocab_size)
* use the same weights for the embedding layer and the pre-softmax layer; dimension 128
* train with Adam (using default learning rates) for at least 20 epochs


In [73]:
class LSTMNetwork(nn.Module):
    # a PyTorch Module that holds the neural network for your model

    def __init__(self):
        super().__init__()

        self.lstm = nn.LSTM(128, 512, num_layers = 3, batch_first = True, dropout = 0.5)
        self.linear1 = nn.Linear(512,128)
        self.linear2 = nn.Linear(128,vocab_size)


    def forward(self, x, state):
        """Compute the output of the network.
        
        Note: In the Pytorch LSTM tutorial, the state variable is named "hidden":
        https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

        The torch.nn.LSTM documentation is quite helpful:
        https://pytorch.org/docs/stable/nn.html#lstm
    
        x - a tensor of int64 inputs with shape (seq_len, batch)
        state - a tuple of two tensors with shape (num_layers, batch, hidden_size)
                representing the hidden state and cell state of the LSTM.
        returns a tuple with two elements:
          - a tensor of log probabilities with shape (seq_len, batch, vocab_size)
          - a state tuple returned by applying the LSTM.
        """

        # Note that the nn.LSTM module expects inputs with the sequence 
        # dimension before the batch by default.
        # In this case the dimensions are already in the right order, 
        # but watch out for this since sometimes people put the batch first
        x = x.type(torch.LongTensor).cuda()
        x = F.embedding(x, self.linear2.weight)
        x, state = self.lstm(x, state)
        x = F.dropout(x)
        x = self.linear1(x)
        x = self.linear2(x)
        x = F.softmax(x)
        return x.view((64,-1,vocab_size)), state

class LSTMModel:
    "A class that wraps LSTMNetwork to handle training and evaluation."

    def __init__(self):
        self.network = LSTMNetwork().cuda()

    def train(self):
        train_iterator = torchtext.data.BPTTIterator(train_dataset, batch_size=64, 
                                                     bptt_len=32, device='cuda')
        # Iterate over train_iterator with a for loop to get batches
        # each batch object has a .text and .target attribute with
        # token id tensors for the input and output respectively.

        # The initial state passed into the LSTM should be set to zero.

        self.network.cuda()
        learning_rate = 1e-3
        optimizer = torch.optim.Adam(self.network.parameters(), lr=learning_rate)
    
        self.network.train()
        validation_score = 0
        state = (torch.zeros(3,32,512).cuda(), torch.zeros(3,32,512).cuda())

        for epoch in range(20):
            print('Epoch', epoch)
            for batch in tqdm.tqdm_notebook(train_iterator, leave = False):
                text, target = batch.text, batch.target
                assert self.network.training, 'make sure network is in train mode'

                optimizer.zero_grad()
                text = text.cuda()
                if text.shape[0] < 32:
                    break #problems with hidden state shapes
                target = target.cuda()
                pred, state = self.network(text, state)
                h,c = state
                state = (h.detach(), c.detach())
                pred = pred.view((64,vocab_size,-1))
                target = target.view((64,-1))
                loss = F.nll_loss(pred, target)
                loss.backward()
                optimizer.step()

            if epoch == 0:
                self.state = state

            val = self.dataset_perplexity(validation_dataset)
            self.network.train()
            print('Validation score:', val)

            if val > validation_score:
                validation_score = val
                torch.save(self.network.state_dict(), os.path.join(os.getcwd(), 'best_model_lstm'))
                self.state = state

        self.network.load_state_dict(torch.load(os.path.join(os.getcwd(), 'best_model_lstm')))

    def next_word_probabilities(self, text_prefix):
        "Return a list of probabilities for each word in the vocabulary."

        prefix_token_tensor = torch.tensor(ids(text_prefix), device='cuda').view(-1, 1)
        
        self.network.eval()
        probas, _ = self.network(prefix_token_tensor, self.state)
        probas.detach().cpu().squeeze()
        return probas

    def dataset_perplexity(self, torchtext_dataset):
        "Return perplexity as a float."
        # Your code should be very similar to next_word_probabilities, but
        # run in a loop over batches. Use torch.no_grad() for extra speed.
        with torch.no_grad():
            iterator = torchtext.data.BPTTIterator(torchtext_dataset, batch_size=64, bptt_len=32, device='cuda')

            self.network.eval()

            log_probabilities = []
            for entry in tqdm.tqdm_notebook(iterator):
                text, target = entry.text, entry.target
                text = text.cuda()
                target = target.cuda()
                state = self.state
                if text.shape[0] < 32:
                    h,c = state
                    state = (h[:,:text.shape[0],:].contiguous(),c[:,:text.shape[0],:].contiguous())
                probas, _ = self.network(text, state)
                probas = probas.squeeze().detach().cpu().numpy()
                target = target.detach().cpu().numpy()
                probas = probas.reshape((-1,vocab_size))
                probas = probas[np.arange(len(probas)), target.ravel()].reshape(1,-1)
                log_probabilities += list(np.log2(probas))
            log_probabilities = np.concatenate(tuple(log_probabilities))
            return 2 ** -np.mean(log_probabilities)

lstm_model = LSTMModel()

lstm_model.train()

print('lstm validation perplexity:', lstm_model.dataset_perplexity(validation_dataset))
save_truncated_distribution(lstm_model, 'lstm_predictions.npy')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


RuntimeError: ignored

Now it's time for you to experiment.  Try to reach a validation perplexity below 120.

It is okay if the bulk of your improvements are due to hyperparameter tuning (such as changing number or sizes of layers), but implement at least one more substantial change to the model.  Here are some ideas (several of which come from https://arxiv.org/pdf/1708.02182.pdf):
* activation regularization - add a l2 regularization penalty on the activation of the LSTM output (standard l2 regularization is on the weights)
* weight-drop regularization - apply dropout to the weight matrices instead of activations
* learning rate scheduling - decrease the learning rate during training
* embedding dropout - zero out the entire embedding for a random set of words in the embedding matrix
* ensembling - average the predictions of several models trained with different initialization random seeds
* temporal activation regularization - add l2 regularization on the difference between the LSTM output activations at adjacent timesteps

You may notice that most of these suggestions are regularization techniques.  This dataset is considered fairly small, so regularization is one of the best ways to improve performance.

In the block below, describe the changes you made and how they affected performance.  If a change doesn't help, don't include it in the model you submit, but write about it here.

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

For some reason my perplexity calculation is completely off...

LSTM validation perplexity before your extra exploration: ***fill in here***

Final validation perplexity: ***fill in here***

### Submission

Upload a submission with the following files to Gradescope:
* proj_1.ipynb (rename to match this exactly)
* lstm_predictions.npy (this should also include all improvements from your exploration)
* neural_trigram_predictions.npy
* trigram_kn_predictions.npy
* bigram_predictions.npy

You can upload files individually or as part of a zip file, but if using a zip file be sure you are zipping the files directly and not a folder that contains them.

Be sure to check the output of the autograder after it runs.  It should confirm that no files are missing and that the output files have the correct format.  Note that the test set perplexities shown by the autograder are on a completely different scale from your validation set perplexities due to truncating the distribution and selecting different text.  Don't worry if the values seem much worse.